In [1]:
import tweepy
import csv
# utilities
import re
import numpy as np
import pandas as pd
# plotting
import seaborn as sns
import matplotlib.pyplot as plt
# nltk
from nltk.stem import WordNetLemmatizer
# sklearn
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report


####input your credentials here
consumer_key = 'VqsqhdXUUZyvHHSRHNrxX6KkO'
consumer_secret = 'IomPXr1FlzSg2Dze4JMFtmIRQ60edmTDhhoFF1D8SyEc0Uh9SX'
access_token = '528704794-BQCDkitN1jTfSl46Hr7RcCKmNOUqQprB2vndvAql'
access_token_secret = 'eAqnGMLioVl342jSBN2DjxWp0zs5ly483XdjN2QNPZrSA'


def twitter_path():
    """
    A function to connect to twitter
    """
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)
    return api

extract = twitter_path()

tweets = extract.search_tweets(q=["Kanye","Kim"], count = 300, lang="en", possibly_sensitive=True)
print("Number of tweets extracted: {}.\n".format(len(tweets)))

print("Recent tweets: \n")
for tweet in tweets:
    print(tweet.text)
    print()

ModuleNotFoundError: No module named 'tweepy'

In [ ]:
#data = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])
data1 = pd.Series(data=[tweet.text for tweet in tweets])

In [ ]:
#create dataframe from series
df = pd.DataFrame(data1.values.tolist(), index=data1.index)
df.columns = ["tweet"]
df.head()

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

#Tokenizing words for patterns
regexp = RegexpTokenizer('\w+')
df['text_token'] = df['tweet'].apply(regexp.tokenize)

#Removing stopwords and infrequent words
stopwords = nltk.corpus.stopwords.words("english")
remove = ['https','the','this']
stopwords.extend(remove)

df['text_token'] = df['text_token'].apply(lambda x: [item for item in x if not isinstance(item, int)])
df['text_token'] = df['text_token'].apply(lambda x: [item for item in x if item not in stopwords])
df['text_token'] = df['text_token'].apply(lambda x: ' '.join([item for item in x if len(item)>2]))
df['text_token'] = df['text_token'].apply(lambda x: " ".join(x.lower() for x in x.split()))

#Lemmatization: grouping inflected words 
wnl = WordNetLemmatizer()
df['text_token'] = df['text_token'].apply(wnl.lemmatize)
df.head()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

#Collecting all the words from token text
all_words = ' '.join([word for word in df['text_token']])

words = nltk.word_tokenize(all_words)
fd = FreqDist(words)

In [ ]:
fd.most_common(5)

In [ ]:
fd.tabulate(5)

In [ ]:
nltk.download('vader_lexicon')

from nltk.sentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

df['polarity'] = df['text_token'].apply(lambda x: analyzer.polarity_scores(x))

df = pd.concat(
    [df.drop(['polarity'], axis=1),
    df['polarity'].apply(pd.Series)], axis=1)
df.head(10)

In [ ]:
#adding sentiment
df['sentiment'] = df['compound'].apply(lambda x: 'positive' if x >0 else 'neutral' if x==0 else 'negative')

df.head(50)

In [ ]:
import seaborn as sns

# Number of tweets 
sns.countplot(y='sentiment', 
             data=df, 
             palette=['#b2d8d8',"#008080", '#db3d13']
             );

In [ ]:
df = df[~df.sentiment.str.contains("neutral")]
df = df[['text_token','sentiment']]

In [ ]:
df['sentiment'][df['sentiment']=='negative']=0
df['sentiment'][df['sentiment']=='positive']=1
df

In [ ]:
def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)
df['text_token'] = df['text_token'].apply(lambda x: cleaning_numbers(x))
df['text_token'].tail()

In [ ]:
data_pos = df[df['sentiment'] == 1]
data_neg = df[df['sentiment'] == 0]
data = pd.concat([data_pos, data_neg])
data['sentiment'] = data['sentiment'].astype('int')

In [ ]:
#token = RegexpTokenizer(r'\w+')
#data['text_token'] = data['text_token'].apply(token.tokenize)
#data

In [ ]:
#Transforming Dataset and creating train/test arrays
#TfidfVectorizer highlights word frequency scores of words that 
#are more interesting

from sklearn.model_selection import train_test_split

X = data.text_token
y = data.sentiment

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.7, random_state=2)

vectoriser = TfidfVectorizer(ngram_range=(1,2), max_features=500000, lowercase=False)
vectoriser.fit(X_train)
print('No. of feature_words: ', len(vectoriser.get_feature_names()))



In [ ]:
X_train = vectoriser.transform(X_train)
X_test  = vectoriser.transform(X_test)

In [ ]:
def model_Evaluate(model):
    # Predict values for Test dataset
    y_pred = model.predict(X_test)
    # Print the evaluation metrics for the dataset.
    print(classification_report(y_test, y_pred))
    # Compute and plot the Confusion matrix
    cf_matrix = confusion_matrix(y_test, y_pred)
    categories = ['Negative','Positive']
    group_names = ['True Neg\n','False Pos\n', 'False Neg\n','True Pos\n']
    group_percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]
    labels = [f'{v1}{v2}' for v1, v2 in zip(group_names,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)
    sns.heatmap(cf_matrix, annot = labels, cmap = 'Blues',fmt = '',
    xticklabels = categories, yticklabels = categories)
    plt.xlabel("Predicted values", fontdict = {'size':14}, labelpad = 10)
    plt.ylabel("Actual values" , fontdict = {'size':14}, labelpad = 10)
    plt.title ("Confusion Matrix", fontdict = {'size':18}, pad = 20)

In [ ]:
LRmodel = LogisticRegression(C = 2, max_iter = 1000, n_jobs=-1)
LRmodel.fit(X_train, y_train)
model_Evaluate(LRmodel)
y_pred3 = LRmodel.predict(X_test)

In [ ]:
#create dataframe from series
#df = pd.DataFrame(data1.values.tolist(), index=data1.index)
#df.columns = ["tweet"]

#Create stopword list and create a new column 
#stop = stopwords.words('english')
#pat = r'\b(?:{})\b'.format('|'.join(stop))
#df['tweet_without_stopwords'] = df['tweet'].str.replace(pat, '')
#df['tweet_without_stopwords'] = df['tweet_without_stopwords'].str.replace(r'\s+', ' ')
#df['tweet_without_stopwords'] = df['tweet_without_stopwords'].apply(lambda x: " ".join(x.lower() for x in x.split()))

#revert dataframe back to series and split arrays into columns
#df1 = df[['tweet_without_stopwords']].squeeze('columns').str.split()
#data = pd.DataFrame(df1.values.tolist(), index=df1.index)
#data.head()
#df

In [ ]:
new_series = df.squeeze(axis=0)
print(type(new_series))

In [ ]:
print(dir(tweets[0]))

In [ ]:
# We print info from the first tweet:
print(tweets[0].id)
print(tweets[0].created_at)
print(tweets[0].source)
print(tweets[0].favorite_count)
print(tweets[0].retweet_count)
print(tweets[0].geo)
print(tweets[0].coordinates)
print(tweets[0].entities)

In [ ]:
# Open/Create a file to append data
csvFile = open('ua.csv', 'a')
#Use csv Writer
csvWriter = csv.writer(csvFile)

#for tweet in tweepy.Cursor(api.search,q="#Joker",count=100,
 #                          lang="en",
  #                         since="2018-04-07").items():
   # print (tweet.created_at, tweet.text)
    #csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])

#for tweets in api.search_tweets(q="Ukraine", lang="en", possibly_sensitive=True):
    #print(tweets.text)

In [ ]:
def save_json(file_name, file_content):
    with open(path + file_name, 'w', encoding='utf-8') as f:
        json.dump(file_content, f, ensure_ascii=False, indent=4)